In [ ]:
from itertools import product

import numpy as np
from aocd.models Puzzle
from aocd.models import product

In [9]:
year, day = 2024, 6

In [10]:
puzzle = Puzzle(year=year, day=day)

In [11]:
puzzle.examples[0].answer_a

'41'

In [12]:
example = puzzle.examples[0].input_data
print(example)

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...


In [32]:
def solution_a(data) -> int:
    # The map shows the current position of the guard with ^ 
    # (to indicate the guard is currently facing up from the perspective of the map).
    # Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.
    # Rules: the guard always moves forward, until it hits an obstruction.
    # Then the guard turns right, and moves forward until it hits an obstruction.
    grid = np.array([list(m) for m in data.splitlines()])
    n, c = grid.shape
    print("grid size: ", n, c)
    guard_pos = np.argwhere(grid == '^')[0]
    
    facing_dir = 0 # degrees, 0 is up, 90 is right, 180 is down, 270 is left
    
    # check if guard is inside grid
    def is_inside(pos):
        return 0 <= pos[0] < n and 0 <= pos[1] < c
    
    while is_inside(guard_pos):
        grid[tuple(guard_pos)] = "X"
        
        # move guard
        next_pos = guard_pos.copy()
        if facing_dir == 0:
            next_pos[0] -= 1
        elif facing_dir == 90:
            next_pos[1] += 1
        elif facing_dir == 180:
            next_pos[0] += 1
        elif facing_dir == 270:
            next_pos[1] -= 1
            
        if not is_inside(next_pos):
            break
            
        # check if guard is facing an obstruction
        if grid[tuple(next_pos)] == '#':
            facing_dir = (facing_dir + 90) % 360
        else:
            guard_pos = next_pos
            
    steps = np.sum(grid == "X")
        
    return grid, steps

In [33]:
solution_a(puzzle.examples[0].input_data)

grid size:  10 10


(array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', 'X', 'X', 'X', 'X', 'X', '#'],
        ['.', '.', '.', '.', 'X', '.', '.', '.', 'X', '.'],
        ['.', '.', '#', '.', 'X', '.', '.', '.', 'X', '.'],
        ['.', '.', 'X', 'X', 'X', 'X', 'X', '#', 'X', '.'],
        ['.', '.', 'X', '.', 'X', '.', 'X', '.', 'X', '.'],
        ['.', '#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.'],
        ['.', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '#', '.'],
        ['#', 'X', 'X', 'X', 'X', 'X', 'X', 'X', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', 'X', '.', '.']], dtype='<U1'),
 np.int64(41))

In [37]:
assert solution_a(puzzle.examples[0].input_data)[1] == int(puzzle.examples[0].answer_a)

grid size:  10 10


In [38]:
answer_a = solution_a(puzzle.input_data)
answer_a = answer_a[1]

grid size:  130 130


In [39]:
answer_a

np.int64(4758)

In [40]:
puzzle.answer_a = answer_a

coerced int64 value np.int64(4758) for 2024/06 to '4758'


That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


## Part Two


In [41]:
print(puzzle.input_data)

#.....#.........#.........................#...#....................#......#.......#...........#........#..........................
..................#......#..................................#..................#....#...............#.#....................#.....#
..............#...........#......#.#.......................................#..................#......#............................
##...............................#..........#...................#.......#................................................#........
#.....................#.......##................#...#..........#............................................#...#....#............
........#........#.............#...............#...##...................#...........#...........#.......#...........##.#...#.#....
................................................#....................................................#.#..........................
.........#....#....................................................................

In [ ]:
def solution_b(data) -> int:
    # The map shows the current position of the guard with ^
    # (to indicate the guard is currently facing up from the perspective of the map).
    # Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.
    # Rules: the guard always moves forward, until it hits an obstruction.
    # Then the guard turns right, and moves forward until it hits an obstruction.
    grid = np.array([list(m) for m in data.splitlines()])
    n, c = grid.shape
    print("grid size: ", n, c)
    guard_pos = np.argwhere(grid == "^")[0]

    facing_dir = 0  # degrees, 0 is up, 90 is right, 180 is down, 270 is left

    # check if guard is inside grid
    def is_inside(pos):
        return 0 <= pos[0] < n and 0 <= pos[1] < c
    
    def get_guard_path(grid, pos, facing_dir):
        grid = grid.copy()
        while is_inside(pos):
            grid[tuple(pos)] = "X"
            
            # move guard
            next_pos = pos.copy()
            if facing_dir == 0:
                next_pos[0] -= 1
            elif facing_dir == 90:
                next_pos[1] += 1
            elif facing_dir == 180:
                next_pos[0] += 1
            elif facing_dir == 270:
                next_pos[1] -= 1
                
            if not is_inside(next_pos):
                break
                
            # check if guard is facing an obstruction
            if grid[tuple(next_pos)] == '#':
                facing_dir = (facing_dir + 90) % 360
            else:
                pos = next_pos
                
        return grid

    def has_loop(grid, pos, facing_dir, obstacle):
        if grid[tuple(obstacle)] == "#" or tuple(obstacle) == tuple(pos):
            return False
        
        grid = grid.copy()
        grid[tuple(obstacle)] = "#"
        n, c = grid.shape
        longest_path = []
        while is_inside(pos):
            grid[tuple(pos)] = "X"

            # move guard
            next_pos = pos.copy()
            if facing_dir == 0:
                next_pos[0] -= 1
            elif facing_dir == 90:
                next_pos[1] += 1
            elif facing_dir == 180:
                next_pos[0] += 1
            elif facing_dir == 270:
                next_pos[1] -= 1

            if not is_inside(next_pos):
                #print(f"guard is outside grid at {pos}")
                #print(grid)
                return False

            # check if guard is facing an obstruction
            if grid[tuple(next_pos)] == "#":
                facing_dir = (facing_dir + 90) % 360
            elif grid[tuple(next_pos)] == "X":
                if tuple(next_pos) in longest_path:
                    print(f"loop detected for obstacle at {obstacle}")
                    return True
                else:
                    longest_path.append(tuple(next_pos))
                    pos = next_pos
            else:
                longest_path = []
                pos = next_pos
                
    guard_path = get_guard_path(grid, guard_pos, facing_dir)
    possible_obstacle_pos = np.argwhere(guard_path == "X")
    
    for obstacle in possible_obstacle_pos:
        if has_loop(grid, guard_pos, facing_dir, obstacle):
            grid[tuple(obstacle)] = "O"
            
    return grid, np.sum(grid == "O")


In [94]:
solution_b(puzzle.examples[0].input_data)

grid size:  10 10
loop detected for onstacle at [6 3]
loop detected for onstacle at [7 6]
loop detected for onstacle at [7 7]
loop detected for onstacle at [8 1]
loop detected for onstacle at [8 3]
loop detected for onstacle at [9 7]


(array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '#', '.', 'O', '^', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', 'O', 'O', '#', '.'],
        ['#', 'O', '.', 'O', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', 'O', '.', '.']], dtype='<U1'),
 np.int64(6))

In [95]:
assert solution_b(puzzle.examples[0].input_data)[1] == 6

grid size:  10 10
loop detected for onstacle at [6 3]
loop detected for onstacle at [7 6]
loop detected for onstacle at [7 7]
loop detected for onstacle at [8 1]
loop detected for onstacle at [8 3]
loop detected for onstacle at [9 7]


In [ ]:
answer_b = solution_b(puzzle.input_data)
answer_b

grid size:  130 130
loop detected for onstacle at [ 3 76]
loop detected for onstacle at [ 3 77]
loop detected for onstacle at [ 3 79]
loop detected for onstacle at [ 3 84]
loop detected for onstacle at [ 3 87]
loop detected for onstacle at [ 3 88]
loop detected for onstacle at [ 3 89]
loop detected for onstacle at [ 3 94]
loop detected for onstacle at [  3 100]
loop detected for onstacle at [  3 101]
loop detected for onstacle at [  3 107]
loop detected for onstacle at [  3 110]
loop detected for onstacle at [  3 114]
loop detected for onstacle at [  3 115]
loop detected for onstacle at [ 4 75]
loop detected for onstacle at [ 5 75]
loop detected for onstacle at [ 6 75]
loop detected for onstacle at [ 7 59]
loop detected for onstacle at [ 8 75]
loop detected for onstacle at [ 9 46]
loop detected for onstacle at [ 9 47]
loop detected for onstacle at [ 9 51]
loop detected for onstacle at [ 9 55]
loop detected for onstacle at [ 9 58]
loop detected for onstacle at [ 9 59]
loop detected for 

In [ ]:
answer_b = answer_b[1]

In [ ]:
puzzle.answer_b = answer_b

aocd will not submit that answer again. At 2024-12-05 15:16:52.203400-05:00 you've previously submitted 5817 and the server responded with:
That's not the right answer; your answer is too low.  If you're stuck, make sure you're using the full input data; there are also some general tips on the about page, or you can ask for hints on the subreddit.  Please wait one minute before trying again. [Return to Day 5]
